## Cost Function and Backpropagation

- **Cost Function**

    - Let's first define a few variables that we will need to use:

        - L = total number of layers in the network

        - $s_l$ = number of units (not counting bias unit) in layer l

        - K = number of output units/classes
    - For neural networks, the cost function is going to be slightly more complicated:
    
    $$J(\Theta) = -\dfrac{1}{m}\sum_{i=1}^m\sum_{k=1}^K[y_k^{(i)}log((h_\Theta(x^{(i)})) + (1-y_k^{(i)})log(1-h_\Theta(x^{(i)}))_k)] + \dfrac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}(\Theta_{j,i}^{(l)})^2$$

- **Backpropagation Algorithm**

    - "Backpropagation" is neural-network terminology for minizing our cost function, just like what we were doing with gradient descent in logistic and linear regression. Our goal is to compute: $min_\Theta J(\Theta)$

    - That is, we want to minimize our cost function J using an optimal set of parameters in theta. In this section, we'll look at the equations we use to compute the partial derivative of $J(\Theta)$: $\dfrac{\partial}{\partial\Theta_{i,j}^{(l)}}J(\Theta)$

    - Algorithm:

        **Backpropagation algorithm**

        - Given training set {$(x^{(1)},y^{(1)}),...,(x^{(m)},y^{(m)})$}

        - Set $\Delta_{i,j} := 0$ for all (l,i,j)

        - For training example t = 1 to m:

        1. Set $a^{(1)} := x^{(t)}$

        2. Perform forward propagation to compute $a^{(l)}$ for l=2, 3, ..., L

        3. Using $y^{(t)}$ compute $\delta^{(L)} = a^{(L)} - y^{(t)}$

        Where L is our total number of layers and $a^{(L)}$ is the vector of outputs of the activation units for the last layer. So our "error values" for the last layer are simply the differences of our actual results in the last layer and the correct outputs in y. To get the delta values of the layers before the last layer, we can use an equation that steps us back from right to left:

        4. Compute $\delta^{(L-1)}, \delta^{(L-2)}, ..., \delta^{(2)}$ using $\delta^{(l)} = ((\Theta^{(l)})^T\delta^{(l+1)}).*a^{(l)}.*(1-a^{(l)})$

        The delta values of layer l are calculated by multiplying the delta values in the next layer with the theta matrix of layer l. We then element-wise multiply that with a function called g', or g-prime, which is the derivative of the activation function g evaluated with the input values given by $z^{(l)}$.

        The g-prime derivative terms can also be writtten out as:

        $g'(z^{(l)}) = a^{(l)}.*(1-a^{(l)})$

        5. $\Delta_{i,j}^{(l)} := \Delta_{i,j}^{(l)} + a_j^{(l)}\delta_j^{(l+1)}$ or with vectorization, $\Delta^{(l)} := \Delta^{(l)} + \delta^{(l+1)}(a^{(l)})^T$

        Hence we update our new $\Delta$ matrix

        - $D_{i,j} := \dfrac{1}{m}(\Delta_{i,j}^{(l)} + \lambda\Theta_{i,j}^{(l)})$, if j!=0

        - $D_{i,j} := \dfrac{1}{m}\Delta_{i,j}^{(l)}$, if j=0

        The capital-delta matrix D is used as an "accumulator" to add up our values as we go along and eventually compute our partial derivative. Thus we get $\frac{\partial}{\partial\Theta_{i,j}^{(l)}} = D_{i,j}^{(l)}$

- **Backpropagation Intuition**

    - Intuitively, $\delta_j^{(l)}$ is the "error" for $a_j^{(l)}$ (unit j in layer l). More formally, the delta values are actually the derivative of the cost function:

    $\delta_j^{(l)} = \dfrac{\partial}{\partial z_j^{(l)}}cost(t)$

## Backpropagation in Practice

- **Implementation Note: Unrolling Parameters**

    - With neural networks, we are working with sets of matrices:

    $\Theta^{(1)}$, $\Theta^{(2)}$, $\Theta^{(3)}$, ...

    $D^{(1)}$, $D^{(2)}$, $D^{(3)}$, ...

    - In order to use optimizing functions such as "fminunc()", we will wnat to "unroll" all the elements and put them into long vector:

        ```code
        thetaVec = [Theta1(:); Theta2(:); Theta3(:)]
        deltaVec = [ D1(:); D2(:); D3(:)]
        ```
    - Then we can get back our original matrices from the "unrolled" versions. Example, if dimensions of Theta1 is 10x11, Theta2 is 10x11 and Theta3 is 1x11, we have:

        ```code
        Theta1 = reshape(ThetaVec(1:110),10,11)
        Theta2 = reshape(ThetaVec(111:220),10,11)
        Theta33 = reshape(ThetaVec(221,231),1,11)
        ```
    - Summarize: Learning Algorithm:

        - Have initial parameters $\Theta^{(1)}$, $\Theta^{(2)}$, $\Theta^{(3)}$

        - Unroll to get initialTheta to pass to fminunc(@costFunction, initialTheta, options)

        - function \[jVal, gradVec\] = costFunction(thetaVec)
        
            From thetaVec, get $\Theta^{(1)}$, $\Theta^{(2)}$, $\Theta^{(3)}$ by reshape

            Use forward prop/back prop to compute $D^{(1)}$, $D^{(1)}$, $D^{(1)}$ and $J(\Theta)$

            Unroll $D^{(1)}$, $D^{(1)}$, $D^{(1)}$ to get gradVec

- **Gradient Checking**
    
    - Gradient checking will assure that our backproppagation works as intended. We can approximate the derivative of our cost function with:

    $\dfrac{\partial}{\partial\Theta}J(\Theta) \approx \dfrac{J(\Theta + \epsilon) - J(\Theta - \epsilon)}{2\epsilon}$

    - With multiply theta matrices, we can approximate the derivative **with respect to $\Theta_j$** as follows"

    $\dfrac{\partial}{\partial\Theta}J(\Theta) \approx \dfrac{J(\Theta_1, ..., \Theta_j + \epsilon, ..., \Theta_n) - J(\Theta_1, ..., \Theta_j - \epsilon, \Theta_n)}{2\epsilon}$

    - A small value for $\epsilon$ such as $\epsilon = 10^{-4}$, guarantees that the math works out properly.

    - We previously saw how to calculate the deltaVector. So once we compute our gradApprox vector, we can check that gradApprox $\approx$ deltaVector. After verified that your backproppagation algorithm is correctt, you don't need to compute gradApprox again.

- **Random Initialzation**

    - Initialzing all theta weights to zero doesn't work with neural networks. Instead we can randomly initialize our weights for our $\Theta$ matrices using the following method:

        Initialzation each $\Theta_{i,j}^{(l)}$ to a random value in $[-\epsilon,\epsilon]$

        Theta1 = rand(10,11)*(2*INIT_EPSILON) - INIT_EPSILON;

        Theta2 = rand(1,11)*(2*INIT_EPSILON) - INIT_EPSILON;

- **Summary: Putting It Together**

    - First, pick a network architecture; choose the layout of your neural network, including how many hidden units in each layer and how many layers in total you want to have.
        
        - Number of input units = dimension of features $x^{(i)}$

        - Number of output units = number of classes

        - Number of hidden units per layer = usually more the better (must balance with cost of computation as it increases with more hidden units)

        - Defaults: 1 hidden layer. If you have more than 1 hidden layer, then it is recommended that you have the same number of units in every hidden layer
    
    - Training a Neural network

        1. Ramdomly initialize the weights

        2. Implement forward propagation to get $h_\Theta(x^{(i)})$ for any $x^{(i)}$

        3. Implement the cost function

        4. Implement backproppagation to compute partial derivatives

        5. Use gradient checking to confirm that your backproppagation works. Then disable gradient checking

        6. Use gradient descent or a built-in optimization function to minimize the cost function with the weights in theta

    - When we perform forward and back proppagation, we loop on every training example:

        ```
        for i = 1:m
            Perform forward propagation and backproppagation using example (x(i), y(i))
            (get activations a(l) and delta terms d(l) for l = 2, 3, ..., L)
        ```

        